#### The following examples are taken from the official scikit learn website documentations

In [1]:
import numpy as np;
from sklearn.model_selection import train_test_split;
from sklearn import datasets;
from sklearn import svm;

In [2]:
X, y = datasets.load_iris(return_X_y=True);
print(f"X.shape = {X.shape}, y.shape = {y.shape}");

X.shape = (150, 4), y.shape = (150,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0);
print(f"X_train.shape = {X_train.shape}, y_train.shape = {y_train.shape}");
print(f"X_test.shape = {X_test.shape}, y_test.shape = {y_test.shape}");

X_train.shape = (90, 4), y_train.shape = (90,)
X_test.shape = (60, 4), y_test.shape = (60,)


In [6]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train);
print(f"score = {clf.score(X_test, y_test)}");

score = 0.9666666666666667


In the k-fold CV, the training set is split into k smaller sets. The following procedure is followed for each of the k "folds":

* A model is trained using `k - 1` of the folds as training data;
* The resulting model is validated on the remaining part of the data, it's used as a test set to compute a performance measure such as accuracy.

The performance measure resported by k-fold cross-validation is then the average of the values computed in the loop.

In [8]:
# use cross_val_score
from sklearn.model_selection import cross_val_score;
clf = svm.SVC(kernel='linear', C=1, random_state=42);
scores = cross_val_score(clf, X, y, cv=5);
print(f"scores: {scores}");
print(f"{scores.mean()} accuracy with a standard deviation of {scores.std()}");

scores: [0.96666667 1.         0.96666667 0.96666667 1.        ]
0.9800000000000001 accuracy with a standard deviation of 0.016329931618554516


In [9]:
# The score computed at each CV iteration is the score method of the estimator.
# It is possible to change this by using the scoring parameter:
from sklearn import metrics
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro');
print(f"scores: {scores}");
print(f"{scores.mean()} accuracy with a standard deviation of {scores.std()}");

scores: [0.96658312 1.         0.96658312 0.96658312 1.        ]
0.9799498746867169 accuracy with a standard deviation of 0.016370858765468226


In [13]:
# to use other cross validation strategies, pass a cross validation iterator
from sklearn.model_selection import ShuffleSplit;
#n_samples = X.shape[0];
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0);
print(f"score: {cross_val_score(clf, X, y, cv=cv)}");

score: [0.97777778 0.97777778 1.         0.95555556 1.        ]


In [16]:
# another option is to use an iterable yielding (train, test)
def custom_cv_2folds(X: np.ndarray):
    n = X.shape[0];
    i = 1;
    while i <= 2:
        idx = np.arange(n * (i - 1) / 2, n * i / 2, dtype=int);
        yield idx, idx;
        i += 1;

custom_cv = custom_cv_2folds(X);
print(f"scores: {cross_val_score(clf, X, y, cv=custom_cv)}");

scores: [1.         0.97333333]


The `cross_validate` function differes from `cross_val_score` in two ways:

* It allows specifying mulitple metrics for evaluation.
* It returns a dict containing fit-times, score-times (optional: training scores, fitted estimators, train-test split) also the test score.


In [17]:
# multiple metrics
from sklearn.model_selection import cross_validate;
from sklearn.metrics import recall_score;
scoring = ['precision_macro', 'recall_macro'];
clf = svm.SVC(kernel='linear', C=1, random_state=0);
scores = cross_validate(clf, X, y, scoring=scoring);
sort = sorted(scores.keys());
print(f"calculated scores: {sort}");

calculated scores: ['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']


In [20]:
for key in scores.keys():
    print(f"{key}: {scores[key]}");

fit_time: [0.00700259 0.00470185 0.0043335  0.00391912 0.00380778]
score_time: [0.01511383 0.01601481 0.01365304 0.0119648  0.01062632]
test_precision_macro: [0.96969697 1.         0.96969697 0.96969697 1.        ]
test_recall_macro: [0.96666667 1.         0.96666667 0.96666667 1.        ]


In [24]:
from sklearn.metrics import make_scorer;
scoring = {'prec_macro': 'precision_macro',
           'rec_macro': make_scorer(recall_score, average='macro')};
scores = cross_validate(clf, X, y, scoring=scoring, cv=5, return_train_score=True, return_indices=True);
for key in scores.keys():
    print(f"{key}: {scores[key]}");

fit_time: [0.00530863 0.00141478 0.00136518 0.00177956 0.00130391]
score_time: [0.00910592 0.00429344 0.00429225 0.00521374 0.00429678]
indices: {'train': (array([ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,
        23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
       134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  20,  21,  22,
        23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  

**Obtaining perdictions by cross-validation**

The function `cross_val_predict` has a similar interface to `cross_val_score`, but returns, for each element in the input, the prediction that was obtained for that element when it was in the test set. Only cross-validation strategies that assign all elements to a test set exactly one can be used.